In [3]:
# scrap all players from premier league website
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

browser = webdriver.Chrome('./chromedriver.exe')
browser.get("https://www.premierleague.com")
time.sleep(2)
browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[5]/button[1]').click()
time.sleep(1)
year=2018
browser.get("https://www.premierleague.com/players")
time.sleep(5)
elem = browser.find_element(By.TAG_NAME,"body")
no_of_pagedowns = 100
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    no_of_pagedowns-=1
trs = browser.find_element(By.CLASS_NAME,"dataContainer").find_elements(By.TAG_NAME,"tr")
text_file = open("Players-{year}.txt".format(year=year), "w")
for tr in trs:
    link=tr.find_element(By.CLASS_NAME,"playerName").get_attribute("href")
    text_file.write(link.split("/")[4]+"\n")
text_file.close()

/Users/hokuma/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  import sys


In [1]:
# scrap all stats from premier league website
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By

browser = webdriver.Chrome('./chromedriver')
browser.get("https://www.premierleague.com")
time.sleep(2)
browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[1]/div[5]/button[1]').click()

for year in range(2014,2023):
    text_file = open("Stats-{year}.txt".format(year=str(year)),"w")
    with open("Players-{year}.txt".format(year=str(year)), "r") as ins:
        for line in ins:
            try:
                browser.get('https://www.premierleague.com/players/{player}/player/stats'.format(player=line.strip()))
                time.sleep(1)
                statblocks = browser.find_elements(By.CLASS_NAME,"statsListBlock")
                for statblock in statblocks:
                    stats=statblock.find_elements(By.CLASS_NAME,"normalStat")
                    text_file.write("playerID: "+line)
                    text_file.write("player: "+browser.find_elements(By.CLASS_NAME,"name")[-1].text+"\n")
                    for stat in stats:
                        text_file.write(stat.text+"\n")
                text_file.write("\n")
            except:
                print(line)
    text_file.close()


/Users/hokuma/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [ ]:
# convert to csv of stats
import csv

attributes=['playerID','Name','Clean sheets','Goals Conceded','Tackles','Tackle success %','Last man tackles','Blocked shots','Interceptions','Clearances','Headed Clearance','Clearances off line','Recoveries','Duels won','Duels lost','Successful 50/50s','Aerial battles won','Aerial battles lost','Own goals','Errors leading to goal','Assists','Passes','Passes per match','Big Chances Created','Crosses','Cross accuracy %','Through balls','Accurate long balls','Yellow cards','Red cards','Fouls','Offsides','Goals','Headed goals','Goals with right foot','Goals with left foot','Hit woodwork','Goals per match','Penalties scored','Freekicks scored','Shots','Shots on target','Shooting accuracy %','Big chances missed','Saves','Penalties Saved','Punches','High Claims','Catches','Sweeper clearances','Throw outs','Goal Kicks']
for year in range(2013,2023):
    txt_file=open('./Stats-{year}.txt'.format(year=str(year)),'r')
    csv_file=open('./Stats-{year}.csv'.format(year=str(year)),'w')

    writer=csv.writer(csv_file)
    writer.writerow(attributes)
    writer=csv.DictWriter(csv_file,attributes,restval='0')
    pID=-1
    stat={}
    for line in txt_file:
        if pID==-1 and 'playerID' in line: # init
            pID=line.split()[1].strip()
            pName=txt_file.readline()[8:]
            stat['playerID']=pID
            stat['Name']=pName.strip()
            continue
        if line=='\n': # reset
            writer.writerow(stat)
            pID=-1
            stat={}
            continue
        if pID!=-1 and 'player' in line: # pass
            txt_file.readline()
            continue
        if line.strip() in attributes:
            stat[line.strip()]=txt_file.readline().strip().strip('%')
    txt_file.close()
    csv_file.close()